In [5]:
import os

In [7]:
!pwd

/media/paolo/Volume/matteo/unimi/tesi_master/code/fear_gen/gaze


In [8]:
os.chdir('./gaze/feature_extraction_gaze/')

In [9]:
import nslr_hmm
from my_utils.gaze import split_events, pixels2angles
import numpy as np

In [10]:
parameters = {
    'distance': 0.53,
    'width': 0.531,
    'height': 0.298,
    'x_res': 1920,
    'y_res': 1080,
    'fs': 1000.}

In [11]:
sub_path = '/home/paolo/matteo/matteo/unimi/tesi_master/code/osfstorage-archive/eye/messages/Look001.asc'

In [12]:
with open(sub_path) as f:
    events = f.readlines()

    events = [event for event in events if 'SFIX' and 'SSACC' and 'MSG' and 'ESACC' not in event]
    trial_start_indices = np.where(["START" in ev for ev in events])[0]
    trial_end_indices = np.where(["END" in ev for ev in events])[0]

    sub_scan = []
    for i in range(len(trial_start_indices)):  # for each trial
        start = trial_start_indices[i] + 7
        if i != len(trial_start_indices) - 1:
            end = trial_start_indices[i + 1] - 9
        else:
            end = trial_end_indices[-1] - 4

        current_trial = events[ start : end ]  # Removing starting and ending info
        trial_coor = []
        for event in current_trial:
            if 'EFIX' not in event:
                try:
                    x = float(event.split('\t')[1])
                    y = float(event.split('\t')[2])
                    trial_coor.append([x, y])
                except:
                    pass
            else:
                x = float(event.split('\t')[4])  # in case of fixation end x and y coordinates are in positions 4,5
                y = float(event.split('\t')[5])
                trial_coor.append([x, y])

        trial_coor = np.asarray(trial_coor)
        sub_scan.append(trial_coor)

scanpath= np.asarray(sub_scan)

/tmp/ipykernel_39818/1903251846.py:34: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  scanpath= np.asarray(sub_scan)


In [13]:
gaze_data = scanpath

In [18]:
gaze_data

array([array([[5.1480000e+02, 5.4700000e+02],
              [5.1120000e+02, 5.4200000e+02],
              [5.0680000e+02, 5.9400000e+02],
              [5.1760000e+02, 5.7400000e+02],
              [4.9870000e+02, 5.6000000e+02],
              [6.2320000e+02, 5.3400000e+02],
              [5.6590000e+02, 5.6700000e+02],
              [4.9460000e+02, 5.6600000e+02],
              [6.2710000e+02, 5.7700000e+02],
              [5.9300000e+02, 5.9600000e+02],
              [5.0130000e+02, 5.5300000e+02],
              [4.9390000e+02, 5.3000000e+02],
              [4.8400000e+02, 5.2200000e+02],
              [6.2610000e+02, 5.1900000e+02],
              [9.7150000e+02, 4.9000000e+02],
              [9.2380000e+02, 4.8500000e+02],
              [9.0270000e+02, 4.5500000e+02],
              [9.0030000e+02, 4.9200000e+02],
              [2.1461624e+07, 8.0000000e+01],
              [1.0241000e+03, 5.0700000e+02],
              [1.0858000e+03, 5.3900000e+02],
              [5.5150000e+02, 5.72

In [15]:
fs=1000

In [16]:
n_samples = gaze_data.shape[0]
dur = n_samples / fs
t = np.linspace(0., dur, n_samples)

gaze_data_ang = pixels2angles(gaze_data, parameters['distance'], parameters['width'],
                        parameters['height'], parameters['x_res'], parameters['y_res'])
print('\nStarting NSLR Classification...')
sample_class, segmentation, seg_class = nslr_hmm.classify_gaze(t, gaze_data_ang)
print('...done. Starting CBW Estimation!')
fixations = sample_class == nslr_hmm.FIXATION
sp = sample_class == nslr_hmm.SMOOTH_PURSUIT
saccades = sample_class == nslr_hmm.SACCADE
pso = sample_class == nslr_hmm.PSO
fix = np.logical_or(fixations, sp).astype(bool)  # merge fixations and smooth pursuits
sac = np.logical_or(saccades, pso).astype(bool) #merge saccades and pso
all_fix = split_events(gaze_data, fix)
all_sac = split_events(gaze_data, sac)

ValueError: operands could not be broadcast together with shapes (160,) (2,) 